# Figure 11: Critique Variants A vs B (Table-2 metrics)

Compares scenario A (`sigma_tau` by regime) and scenario B (all sigmas by regime) from `artifacts/comparisons/*.csv`.

In [ ]:
import os, sys
import pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def _find_project_root():
    here = pathlib.Path.cwd().resolve()
    for p in [here, *here.parents]:
        if (p / "src").is_dir():
            return p
    cand = pathlib.Path("/content/econml")
    if (cand / "src").is_dir():
        return cand
    raise RuntimeError("Could not find project root containing src/.")

PROJECT_ROOT = _find_project_root()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

COMPARE_DIR = pathlib.Path(os.environ.get("CRITIQUE_COMPARE_DIR", str(PROJECT_ROOT / "artifacts" / "comparisons")))
combined_path = COMPARE_DIR / "table2_variants_combined.csv"
delta_path = COMPARE_DIR / "table2_variantB_minus_variantA.csv"
print("COMPARE_DIR:", COMPARE_DIR)
print("combined:", combined_path)
print("delta:", delta_path)
if not combined_path.exists() or not delta_path.exists():
    raise FileNotFoundError(
        "Comparison CSV files are missing. Run scripts/compare_uncertainty_variants.py first."
    )

combined = pd.read_csv(combined_path)
delta = pd.read_csv(delta_path)
print("combined rows:", len(combined), "delta rows:", len(delta))
combined.head()

In [ ]:
key_metrics = ["pi_std_pct", "x_std_pct", "r_std_pct", "i_std_pct"]
variants_keep = ["variant_A_sigma_tau", "variant_B_all_sigma"]

df = combined[combined["variant"].isin(variants_keep)].copy()
order = (
    df[["policy", "regime"]]
    .drop_duplicates()
    .sort_values(["policy", "regime"])
)
order_labels = [f"{p}:{r}" for p, r in order.to_records(index=False)]

fig, axes = plt.subplots(2, 2, figsize=(16, 9), constrained_layout=True)
for ax, metric in zip(axes.ravel(), key_metrics):
    piv = (
        df.pivot_table(index=["policy", "regime"], columns="variant", values=metric, aggfunc="mean")
        .reindex(pd.MultiIndex.from_frame(order), fill_value=np.nan)
    )
    x = np.arange(len(piv))
    w = 0.38
    yA = piv.get("variant_A_sigma_tau", pd.Series(index=piv.index, dtype=float)).values
    yB = piv.get("variant_B_all_sigma", pd.Series(index=piv.index, dtype=float)).values
    ax.bar(x - w/2, yA, width=w, label="A: sigma_tau only")
    ax.bar(x + w/2, yB, width=w, label="B: all sigmas")
    ax.set_xticks(x)
    ax.set_xticklabels(order_labels, rotation=45, ha="right")
    ax.set_title(metric)
    ax.grid(alpha=0.25)

axes[0, 0].legend(loc="best")
fig.suptitle("Critique variants A vs B (Table-2 moments)")

out_dir = PROJECT_ROOT / "artifacts" / "figures"
out_dir.mkdir(parents=True, exist_ok=True)
out_path = out_dir / "fig11_critique_variants_table2.png"
fig.savefig(out_path, dpi=160)
print("Saved:", out_path)
plt.show()

In [ ]:
delta_m = delta.set_index(["policy", "regime"])[key_metrics].sort_index()
fig, ax = plt.subplots(figsize=(10, 5), constrained_layout=True)
mat = delta_m.values
im = ax.imshow(mat, aspect="auto")
ax.set_xticks(np.arange(len(key_metrics)))
ax.set_xticklabels(key_metrics, rotation=30, ha="right")
row_labels = [f"{p}:{r}" for p, r in delta_m.index]
ax.set_yticks(np.arange(len(row_labels)))
ax.set_yticklabels(row_labels)
for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
        ax.text(j, i, f"{mat[i, j]:+.2f}", ha="center", va="center", fontsize=8)
ax.set_title("B - A delta by policy/regime")
fig.colorbar(im, ax=ax, shrink=0.9)
out_path2 = (PROJECT_ROOT / "artifacts" / "figures" / "fig11_critique_variants_delta.png")
fig.savefig(out_path2, dpi=160)
print("Saved:", out_path2)
plt.show()